In [1]:
import os
import caffe
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

os.chdir('/home/zhangkj/software/caffe')

In [2]:
caffe.set_device(2)
caffe.set_mode_gpu()

In [3]:
solver = caffe.get_solver('models/VGGNet/coco/SSD_300x300/solver.prototxt')
solver.net.copy_from('models/VGGNet/coco/SSD_300x300/vgg_coco.caffemodel')

In [4]:
solver.net.forward()
solver.net.blobs['label'].data[0, 0, :, 1]

array([ 46.,  46.,  46.,  51.,  50.,  50.,  50.,  50.], dtype=float32)

In [7]:
solver.net.blobs['data'].data[0, 2, 100:106, 100:106]

array([[ 114.,  118.,  125.,  118.,  113.,   76.],
       [ 109.,  125.,  126.,  122.,  121.,  130.],
       [ 120.,  119.,  120.,  108.,  128.,  127.],
       [ 116.,  129.,  119.,  124.,  120.,  120.],
       [ 126.,  124.,  123.,  122.,  128.,  120.],
       [ 126.,  114.,  119.,  121.,  126.,  125.]], dtype=float32)

In [ ]:
# convert (C, H, W) to (H, W, C)
input = np.array(solver.net.blobs['data'].data[0].transpose(1, 2, 0)) + [104, 117, 123]
# convert BGR to RGB
input = input[:, :, (2, 1, 0)] / 255.
plt.imshow(input)

In [ ]:
pytorch_data = []
with open('input.log', 'r') as f:
    for line in f.readlines():
        pytorch_data.append(float(line))
        
img_data = np.array(pytorch_data).reshape(3, 300, 300).transpose(1, 2, 0) + [104, 117, 123]
img_data = img_data[:, :, (2, 1, 0)] / 255.
plt.imshow(img_data)

In [ ]:
print(solver.net.blobs['mbox_loss'].data)
solver.net.blobs['data'].data[...] = np.array(pytorch_data).reshape(1, 3, 300, 300)
solver.net.forward(start='conv1_1')
print(solver.net.blobs['mbox_loss'].data)

In [ ]:
print(solver.net.blobs['label'].data[:, :, :, 3:7])
print(solver.net.blobs['label'].data[:, :, :, 1])

In [ ]:
solver.net.blobs.keys()
solver.net.blobs['mbox_conf'].data.shape

In [ ]:
solver.net.forward()

compare the difference of two data file

In [ ]:
caffe_data = []
pytorch_data = []
with open('pytorch.log', 'r') as f:
    for line in f.readlines():
        pytorch_data.append(float(line))

with open('caffe.log', 'r') as f:
    for line in f.readlines():
        caffe_data.append(float(line))

count = 0
for c_data, p_data in zip(caffe_data, pytorch_data):
    diff = c_data - p_data
    if diff > 0.0001 or diff < -0.0001:
        count += 1
        print(c_data, "pytorch: ", p_data)
print("count: ", count)

In [ ]:
count = 0
with open('output.log', 'r') as f:
    for line in f.readlines():
        label = float(line)
        if label != -1:
            print(label)
            count = count + 1
print('count: ', count)

In [ ]:
import cv2